In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "lampe01" # username from the json file
os.environ['KAGGLE_KEY'] = "0fe21bb583c2cd3230e9d928c96e27f7" # key from the json file
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CirCor"

In [3]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 1.1.1
Uninstalling scikit-learn-1.1.1:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/scikit_learn-1.1.1.dist-info/*
    /usr/local/lib/python3.8/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.8/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.1.1


In [7]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyaudioprocessing 1.1.13 requires scikit-learn==1.1.1; python_version >= "3.8", but you have scikit-learn 1.1.3 which is incompatible.


In [8]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3


In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import random
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [11]:
mdata = pd.read_csv('/content/drive/MyDrive/ReCirCor/training_data.csv')
input_dir = '/content/drive/MyDrive/ReCirCor/training_data/training_data'

In [12]:
#Gets all the txt files
def getfilenames(inputdir):
    Fname = list()
    Fname = sorted([fname for fname in os.listdir(inputdir) if not fname.startswith('.')and fname.endswith('.txt')])
    
    return Fname

In [13]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [14]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

id = list()
fl = list()
for i in range(23):
  id.append(str(i+1))
  fl.append(0)

In [15]:
nlabels = labels
for i in range(77):
  x = random.randint(1,23)
  id.append(str(i+1))
  filepathslist.append(str(x)+'.wav')
  fl.append(0)
  nlabels.append(labels[x-1])

In [16]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['new ID', 'wav filename','flag value', 'labels'])

In [17]:
id

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77']

In [18]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav',
 '1.wav',
 '17.wav',
 '13.wav',
 '9.wav',
 '2.wav',
 '15.wav',
 '20.wav',
 '3.wav',
 '20.wav',
 '14.wav',
 '22.wav',
 '10.wav',
 '1.wav',
 '4.wav',
 '18.wav',
 '11.wav',
 '8.wav',
 '6.wav',
 '22.wav',
 '20.wav',
 '10.wav',
 '12.wav',
 '20.wav',
 '4.wav',
 '16.wav',
 '21.wav',
 '10.wav',
 '4.wav',
 '13.wav',
 '22.wav',
 '1.wav',
 '6.wav',
 '11.wav',
 '14.wav',
 '6.wav',
 '14.wav',
 '11.wav',
 '20.wav',
 '3.wav',
 '21.wav',
 '16.wav',
 '2.wav',
 '17.wav',
 '3.wav',
 '16.wav',
 '22.wav',
 '5.wav',
 '17.wav',
 '11.wav',
 '14.wav',
 '10.wav',
 '19.wav',
 '22.wav',
 '18.wav',
 '7.wav',
 '15.wav',
 '8.wav',
 '23.wav',
 '3.wav',
 '15.wav',
 '17.wav',
 '16.wav',
 '8.wav',
 '23.wav',
 '9.wav',
 '4.wav',
 '16.wav',
 '22.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 

In [19]:
labels

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1]

In [20]:
labels

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1]

In [21]:
#Inserting filename and label values into dataframe
dataframe['new ID'] = id
dataframe['wav filename'] = filepathslist
dataframe['flag value'] = fl
dataframe['labels'] = labels

In [22]:
dataframe

,new ID,wav filename,flag value,labels
0,1,1.wav,0,0
1,2,2.wav,0,0
2,3,3.wav,0,0
3,4,4.wav,0,0
4,5,5.wav,0,0
...,...,...,...,...
95,73,12.wav,0,1
96,74,17.wav,0,1
97,75,13.wav,0,1
98,76,5.wav,0,0


In [23]:
dataframenew = pd.DataFrame(columns = ['new ID', 'wav filename', 'flag value', 'labels'])

In [24]:
import random

In [25]:
#Get only murmur recordings
def getrecmur(inputdir):
    fname = getfilenames(inputdir)
    items = []
    rinfo = []
    recf = []
    labels = []
    recfile = []
    for i in fname:
        i = inputdir + '/' + i
        with open(i, 'r') as f:
            content = f.readlines()
        numloc = int(content[0].split(' ')[1])
        for i in range(numloc):
            for p in range(len(content)):
                t = content[p].split('\n')
                for it in t:
                    if it.startswith('#Murmur:'):
                        tar = it.split(': ')[1]
                        if tar == 'Present':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 1
                          labels.append(l)
                        elif tar == 'Absent':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 0
                          labels.append(l)
    for items in rinfo:
        for i in range(len(items)):
            if items[i].endswith('.wav'):
                recfile = input_dir + '/' + items[i]
        recf.append(recfile)
    return rinfo, recf, labels

rin, ref, label = getrecmur(input_dir)

In [26]:
len(rin)

3007

In [27]:
newID = list()
wavfilename = list()
newlabels = list()
flagvalue = list()
for i in range(100):
  x = random.randint(1,23)
  newID.append(str(i+1))
  wavfilename.append(str(x)+'.wav')
  flagvalue.append(0)
  newlabels.append(labels[x-1])

x = 100
for i in range(900):
  x = x + 1
  wavfilename.append(ref[i])
  newlabels.append(label[i])
  flagvalue.append(1)
  newID.append(str(x))

In [28]:
len(newID)

1000

In [29]:
dataframenew['new ID'] = newID
dataframenew['wav filename'] = wavfilename
dataframenew['flag value'] = flagvalue
dataframenew['labels'] = newlabels
dataframenew

,new ID,wav filename,flag value,labels
0,1,15.wav,0,1
1,2,15.wav,0,1
2,3,16.wav,0,1
3,4,2.wav,0,0
4,5,3.wav,0,0
...,...,...,...,...
995,996,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
996,997,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
997,998,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
998,999,/content/drive/MyDrive/ReCirCor/training_data/...,1,0


In [30]:
dataframe

,new ID,wav filename,flag value,labels
0,1,1.wav,0,0
1,2,2.wav,0,0
2,3,3.wav,0,0
3,4,4.wav,0,0
4,5,5.wav,0,0
...,...,...,...,...
95,73,12.wav,0,1
96,74,17.wav,0,1
97,75,13.wav,0,1
98,76,5.wav,0,0


In [31]:
dataframenew['labels'].value_counts()

0    729
1    271
Name: labels, dtype: int64

In [32]:
X_train = dataframe['new ID']
y_Train = dataframe['labels']

In [33]:
X_test = dataframenew['new ID']
y_Test = dataframenew['labels']

In [34]:
#X_train, X_test, y_Train, y_Test = train_test_split(dataframenew['new ID'], dataframenew['labels'], train_size=0.8, shuffle=True)

In [35]:
X_train

0      1
1      2
2      3
3      4
4      5
      ..
95    73
96    74
97    75
98    76
99    77
Name: new ID, Length: 100, dtype: object

In [36]:
X_test

0         1
1         2
2         3
3         4
4         5
       ... 
995     996
996     997
997     998
998     999
999    1000
Name: new ID, Length: 1000, dtype: object

In [37]:
#Computing Features - 

#Discrete Wavelet Transform
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    return res

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
def computepsd(x):
    freq, psd = signal.welch(x)
    return psd

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 13).T, axis = 0)
    return mfccs

def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    gfcc = np.array(gfcc['audio'][file_name]['features'])
    return gfcc

def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    return rmsen

def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    return zcr

def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    return stats

In [38]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
        
        finalfeatures = np.hstack((finalfeatures, mfccs))
    if dwt:
        dwtr = computeDWTs(x)
        finalfeatures = np.hstack((finalfeatures, dwtr))
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
        finalfeatures = np.hstack((finalfeatures,specen))
    if psd:
        psds = computepsd(x)
        finalfeatures = np.hstack((finalfeatures, psds))
    if gfcc:
        gfccs = computeGFCC(file_name)
        finalfeatures = np.hstack((finalfeatures, gfccs))
    if rmse:
        rmses = computeRMSEn(x)
        finalfeatures = np.hstack((finalfeatures, rmses))
    if zcr:
        zcrs = computeZCR(x)
        finalfeatures = np.hstack((finalfeatures, zcrs))
    if stats:
        stats1 = computestats(x)
        finalfeatures = np.hstack((finalfeatures, stats1))
    return finalfeatures

In [39]:
#loads audio and extracts required features
def load_file_data(file_num, duration=15, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    i = 0
    for file_no in file_num:
        try:
            #sound_file = folder + '/' + file_name
            sfile = int(file_no)
            sound_file = dataframenew['wav filename'][sfile-1]
            print ("load file ", i, sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_no, sound_file)
                y = librosa.util.fix_length(x, input_length)
            feature = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True)
            #dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True
            notfeatures.append(feature)          
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
        i = i+1 
    return notfeatures

In [40]:
x_train = []
x_test = []
x_train = load_file_data(X_train)
x_test = load_file_data(X_test)

Streaming output truncated to the last 5000 lines.
        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_AV_1.wav
load file  289 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_AV_2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_AV_2.wav
load file  290 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_1.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_1.wav
load file  291 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_2.wav
load file  292 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_

In [41]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

In [42]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [43]:
x_test.shape

(1000, 200)

In [44]:
x_train.shape

(100, 200)

In [45]:
y_Train.dtype

dtype('int64')

In [46]:
y_Train = np.asarray(y_Train).astype(np.float32)

In [47]:
y_Test = np.asarray(y_Test).astype(np.float32)

In [48]:
traindataframe = pd.DataFrame(columns = ['x_train ID', 'labels'])
testdataframe = pd.DataFrame(columns = ['x_test ID', 'labels'])
traindataframe['x_train ID'] = X_train
testdataframe['x_test ID'] = X_test
traindataframe['labels'] = y_Train
testdataframe['labels'] = y_Test

In [49]:
testdataframe

,x_test ID,labels
0,1,1.0
1,2,1.0
2,3,1.0
3,4,0.0
4,5,0.0
...,...,...
995,996,0.0
996,997,0.0
997,998,0.0
998,999,0.0


In [50]:
xtrainwavfiles = list()
trainflagval = list()
for i in X_train:
  num = int(i)
  xtrainwavfiles.append(dataframe['wav filename'][num-1])
  trainflagval.append(dataframe['flag value'][num-1])

In [51]:
xtestwavfiles = list()
flagval = list()
for i in X_test:
  num = int(i)
  xtestwavfiles.append(dataframenew['wav filename'][num-1])
  flagval.append(dataframenew['flag value'][num-1])

In [52]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [54]:
pca = PCA(n_components=20)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.fit_transform(x_test)

In [55]:
x_train_pca.shape

(100, 20)

In [56]:
x_test_pca.shape

(1000, 20)

In [57]:
dictofpca = defaultdict(list)

def getpcalist(x):
  for i in range(20):
    item = x[i]
    dictofpca[f"pca_{i}"].append(item)
  return dictofpca

for i in x_train_pca:
  pcanum = getpcalist(i)

testpcadict = defaultdict(list)

def getpcalist2(x):
  for i in range(20):
    item = x[i]
    testpcadict[f"pca_{i}"].append(item)
  return testpcadict

for i in x_test_pca:
  tpca = getpcalist2(i)

In [58]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train_pca, y_Train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [59]:
clf.score(x_test_pca, y_Test)

0.347

In [60]:
y_pred1 = clf.predict(x_test_pca)
print("Classification Report : \n\n", classification_report(y_Test, y_pred1))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.87      0.12      0.22       729
         1.0       0.29      0.95      0.44       271

    accuracy                           0.35      1000
   macro avg       0.58      0.54      0.33      1000
weighted avg       0.71      0.35      0.28      1000



In [61]:
df1 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value':flagval, 'Real Values':y_Test, 'Predicted Values':y_pred1})
df1

,Input,Wav filename,flag value,Real Values,Predicted Values
0,1,15.wav,0,1.0,1.0
1,2,15.wav,0,1.0,1.0
2,3,16.wav,0,1.0,1.0
3,4,2.wav,0,0.0,1.0
4,5,3.wav,0,0.0,1.0
...,...,...,...,...,...
995,996,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
996,997,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
997,998,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
998,999,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0


In [62]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [63]:
clf2 = LinearSVC(penalty='l2', loss='hinge')
clf2.fit(x_train_pca, y_Train)
clf2.score(x_test_pca, y_Test)

0.419

In [64]:
y_pred2 = clf2.predict(x_test_pca)
print("Classification Report : \n\n", classification_report(y_Test, y_pred2))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.76      0.30      0.43       729
         1.0       0.28      0.75      0.41       271

    accuracy                           0.42      1000
   macro avg       0.52      0.52      0.42      1000
weighted avg       0.63      0.42      0.42      1000



In [65]:
df2 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval, 'Real Values':y_Test, 'Predicted Values':y_pred2})
df2

,Input,Wav filename,flag value,Real Values,Predicted Values
0,1,15.wav,0,1.0,1.0
1,2,15.wav,0,1.0,1.0
2,3,16.wav,0,1.0,1.0
3,4,2.wav,0,0.0,1.0
4,5,3.wav,0,0.0,1.0
...,...,...,...,...,...
995,996,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
996,997,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
997,998,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
998,999,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0


In [66]:
clf4 = SGDClassifier(loss = 'hinge').fit(x_train_pca, y_Train)

In [67]:
clf4.score(x_test_pca, y_Test)

0.644

In [68]:
y_pred4 = clf4.predict(x_test_pca)
#y_pred=np.argmax(y_pred, axis=1)
print("Classification Report : \n\n", classification_report(y_Test, y_pred4))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.73      0.82      0.77       729
         1.0       0.27      0.18      0.22       271

    accuracy                           0.64      1000
   macro avg       0.50      0.50      0.49      1000
weighted avg       0.60      0.64      0.62      1000



In [69]:
df4 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval, 'Real Values':y_Test, 'Predicted Values':y_pred4})
df4

,Input,Wav filename,flag value,Real Values,Predicted Values
0,1,15.wav,0,1.0,0.0
1,2,15.wav,0,1.0,0.0
2,3,16.wav,0,1.0,0.0
3,4,2.wav,0,0.0,0.0
4,5,3.wav,0,0.0,0.0
...,...,...,...,...,...
995,996,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
996,997,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
997,998,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
998,999,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0


In [70]:
#param_grid = {'C': [0.1, 0.5, 1, 10,30, 40, 50, 75, 100, 500, 1000],
              #0.01, 0.1, 0.5, 1, 10, 30, 40, 50, 75, 100, 500, 1000
#              'gamma': [0.01, 0.05, 0.07, 0.1, 0.5, 1, 5, 10, 50],
              #0.01, 0.05, 0.07, 0.1, 0.5, 1, 0.75, 0.25, 0.001, 5, 10, 50
#              'kernel': ['rbf', 'poly', 'linear']}

#grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=1, cv=5)
#grid.fit(x_train_pca, y_Train)
#best_params = grid.best_params_
#print(f"Best params: {best_params}")

In [71]:
svmwpca = SVC(C = 30, gamma = 0.07)
svmwpca.fit(x_train_pca, y_Train)
svmwpca.score(x_test_pca, y_Test)

0.271

In [72]:
y_pred = svmwpca.predict(x_test_pca)
print(classification_report(y_Test, y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       729
         1.0       0.27      1.00      0.43       271

    accuracy                           0.27      1000
   macro avg       0.14      0.50      0.21      1000
weighted avg       0.07      0.27      0.12      1000



In [73]:
pcadataframe = pd.DataFrame({'Input':X_train, 'Wav filename':xtrainwavfiles, 'flag value': trainflagval})
#'Labels':y_Train

In [74]:
for i in pcanum:
  pcadataframe[i] = pcanum[i]

In [75]:
y_newt = []
for i in y_Train:
  y_newt.append(int(i))

In [76]:
pcadataframe['Labels'] = y_newt

In [77]:
pcadataframe

,Input,Wav filename,flag value,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,...,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,Labels
0,1,1.wav,0,1.109221,-0.762093,-1.423857,-0.167523,-1.210244,0.545456,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004095,-0.007347,0.048764,0.014632,0.001360,0
1,2,2.wav,0,1.109221,-0.762093,-1.423855,-0.167524,-1.210245,0.545457,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004094,-0.007347,0.048764,0.014632,0.001360,0
2,3,3.wav,0,0.008469,-0.110031,0.044358,1.755571,-0.012118,-0.161699,-0.127300,...,0.057792,-0.020506,0.015283,-0.001347,-0.074384,0.045197,0.029463,0.008201,-0.004861,0
3,4,4.wav,0,-0.058841,0.957617,0.766501,-0.344643,-1.031310,-0.320412,-0.106349,...,-0.016186,0.043630,0.261955,-0.063241,0.133332,0.002204,-0.008712,-0.028490,0.002532,0
4,5,5.wav,0,0.101232,1.040413,0.158836,-0.694453,-0.400978,-0.878776,-0.370163,...,0.131394,-0.045748,-0.100803,0.009853,-0.117070,0.001870,0.042238,-0.076196,-0.019582,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,73,14.wav,0,-0.058841,0.957618,0.766501,-0.344643,-1.031310,-0.320412,-0.106349,...,-0.016186,0.043630,0.261955,-0.063241,0.133332,0.002204,-0.008711,-0.028490,0.002531,1
96,74,10.wav,0,1.109222,-0.762094,-1.423856,-0.167523,-1.210245,0.545457,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004094,-0.007347,0.048764,0.014632,0.001360,1
97,75,19.wav,0,1.109222,-0.762094,-1.423856,-0.167523,-1.210245,0.545457,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004094,-0.007347,0.048764,0.014632,0.001360,1
98,76,22.wav,0,-0.699710,1.390987,0.118653,0.272301,0.424683,1.042611,-0.036385,...,0.001135,0.315419,-0.027473,0.047252,0.002135,0.000886,0.001469,-0.010688,-0.001253,0


In [78]:
len(x_test_pca)

1000

In [79]:
x_test_pca[0]

array([-9.2113955e+03, -4.7229990e+02,  9.0988495e+01,  6.6392227e+01,
       -3.3966286e+01, -3.2354847e+01, -4.1577015e+00,  5.1850563e-01,
       -1.3084058e+01, -2.3695197e+00, -6.1162889e-01,  8.9815211e-01,
       -3.9419825e+00,  2.4779123e-01,  6.3264567e-01, -3.7267047e-01,
       -3.9206287e-01,  3.5733014e-01, -5.6038147e-01,  2.2686328e-01],
      dtype=float32)

In [80]:
testpcadict = defaultdict(list)
def getpcalist(x):
  for i in range(20):
    item = x[i]
    testpcadict[f"pca_{i}"].append(item)
  return testpcadict

In [81]:
for i in x_test_pca:
  tpca = getpcalist(i)

In [82]:
for i in tpca:
  print(i)

pca_0
pca_1
pca_2
pca_3
pca_4
pca_5
pca_6
pca_7
pca_8
pca_9
pca_10
pca_11
pca_12
pca_13
pca_14
pca_15
pca_16
pca_17
pca_18
pca_19


In [83]:
len(tpca['pca_0'])

1000

In [84]:
pcaresult = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval})

for i in tpca:
  pcaresult[i] = tpca[i]

y_newt2 = []
for i in y_Test:
  y_newt2.append(int(i))

pcaresult['Real Values'] = y_newt2

In [85]:
pcaresult['Predicted Values'] = y_pred2

In [86]:
pcaresult

,Input,Wav filename,flag value,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,...,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,Real Values,Predicted Values
0,1,15.wav,0,-9211.395508,-472.299896,90.988495,66.392227,-33.966286,-32.354847,-4.157701,...,-3.941983,0.247791,0.632646,-0.372670,-0.392063,0.357330,-0.560381,0.226863,1,1.0
1,2,15.wav,0,-9211.451172,-472.279358,91.021271,66.380592,-33.968502,-32.355247,-4.157116,...,-3.941985,0.247793,0.632644,-0.372667,-0.392060,0.357332,-0.560376,0.226864,1,1.0
2,3,16.wav,0,-9211.450195,-472.273926,91.011375,66.441353,-33.953075,-32.343594,-4.262553,...,-4.745503,-0.924848,0.056773,0.820014,-0.579889,0.021857,-0.239185,-0.696719,1,1.0
3,4,2.wav,0,-9211.517578,-472.319763,91.022972,66.403282,-34.013618,-32.353210,-4.294962,...,-4.913331,-1.249652,0.532447,-0.144155,-0.713004,-0.079829,0.284195,0.081602,0,1.0
4,5,3.wav,0,-9211.506836,-472.316467,91.023964,66.400665,-34.011551,-32.349419,-4.276633,...,-5.369754,-0.292552,-0.043860,0.358119,-0.417493,-0.025410,0.482843,0.501281,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,/content/drive/MyDrive/ReCirCor/training_data/...,1,-8048.481445,-509.298157,-21.600298,61.710754,-39.803764,-25.705008,-4.605165,...,-0.529201,0.274869,0.605167,-0.647536,0.167647,-0.044061,-0.507736,-0.184646,0,0.0
996,997,/content/drive/MyDrive/ReCirCor/training_data/...,1,-9015.367188,-465.789734,93.481453,65.570030,-34.947781,-31.853811,-4.414657,...,1.379969,-0.894624,1.449358,-0.280645,-0.549355,1.116954,0.351387,-0.167116,0,1.0
997,998,/content/drive/MyDrive/ReCirCor/training_data/...,1,-5440.315430,-406.941162,108.635071,77.539505,-53.207691,-20.298311,-7.304301,...,0.516232,-0.703933,0.389321,0.410735,0.629932,0.621627,-0.246704,0.184589,0,0.0
998,999,/content/drive/MyDrive/ReCirCor/training_data/...,1,-9004.016602,-464.606903,62.232239,33.672405,-4.054645,-12.174980,-2.914800,...,-1.704483,1.055449,0.792506,-1.013865,0.212860,-0.224098,0.197795,-0.162798,0,1.0


In [98]:
pcadataframe.to_csv('reptrainpca-1.csv')
pcaresult.to_csv('reptestpcalinearsvc-1.csv')

In [88]:
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

pipe_steps = [('scaler', MinMaxScaler()), ('pca', PCA()), ('SVM', SVC(kernel='rbf'))]

check_params= {
    'pca__n_components': [20], 
    'SVM__C': [0.1, 0.5, 1, 10,30, 40, 50, 75, 100, 500, 1000], 
    'SVM__gamma' : [0.01, 0.05, 0.07, 0.1, 0.5, 1, 5, 10, 50]
}

pipeline = Pipeline(pipe_steps)

for cv in tqdm(range(4,11)):
    create_grid = GridSearchCV(pipeline, param_grid=check_params, cv=cv)
    create_grid.fit(x_train, y_Train)
    print("score for %d fold CV -  %3.2f" %(cv, create_grid.score(x_test, y_Test)))
    print(" ")
    print(" ")
    print (create_grid.best_params_)

  0%|          | 0/7 [00:00<?, ?it/s]

score for 4 fold CV -  0.26
 
 
{'SVM__C': 1, 'SVM__gamma': 0.07, 'pca__n_components': 20}
score for 5 fold CV -  0.26
 
 
{'SVM__C': 1, 'SVM__gamma': 0.1, 'pca__n_components': 20}
score for 6 fold CV -  0.27
 
 
{'SVM__C': 0.1, 'SVM__gamma': 0.01, 'pca__n_components': 20}
score for 7 fold CV -  0.27
 
 
{'SVM__C': 0.1, 'SVM__gamma': 0.01, 'pca__n_components': 20}
score for 8 fold CV -  0.27
 
 
{'SVM__C': 0.1, 'SVM__gamma': 0.01, 'pca__n_components': 20}
score for 9 fold CV -  0.27
 
 
{'SVM__C': 0.1, 'SVM__gamma': 0.01, 'pca__n_components': 20}
score for 10 fold CV -  0.27
 
 
{'SVM__C': 0.1, 'SVM__gamma': 0.01, 'pca__n_components': 20}


In [96]:
svmwpca2 = SVC(C = 30, gamma = 0.05)
svmwpca2.fit(x_train_pca, y_Train)
svmwpca2.score(x_test_pca, y_Test)

0.271

In [97]:
y_pred2 = svmwpca2.predict(x_test_pca)
print(classification_report(y_Test, y_pred2))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       729
         1.0       0.27      1.00      0.43       271

    accuracy                           0.27      1000
   macro avg       0.14      0.50      0.21      1000
weighted avg       0.07      0.27      0.12      1000



In [91]:
pcadataframe

,Input,Wav filename,flag value,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,...,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,Labels
0,1,1.wav,0,1.109221,-0.762093,-1.423857,-0.167523,-1.210244,0.545456,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004095,-0.007347,0.048764,0.014632,0.001360,0
1,2,2.wav,0,1.109221,-0.762093,-1.423855,-0.167524,-1.210245,0.545457,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004094,-0.007347,0.048764,0.014632,0.001360,0
2,3,3.wav,0,0.008469,-0.110031,0.044358,1.755571,-0.012118,-0.161699,-0.127300,...,0.057792,-0.020506,0.015283,-0.001347,-0.074384,0.045197,0.029463,0.008201,-0.004861,0
3,4,4.wav,0,-0.058841,0.957617,0.766501,-0.344643,-1.031310,-0.320412,-0.106349,...,-0.016186,0.043630,0.261955,-0.063241,0.133332,0.002204,-0.008712,-0.028490,0.002532,0
4,5,5.wav,0,0.101232,1.040413,0.158836,-0.694453,-0.400978,-0.878776,-0.370163,...,0.131394,-0.045748,-0.100803,0.009853,-0.117070,0.001870,0.042238,-0.076196,-0.019582,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,73,14.wav,0,-0.058841,0.957618,0.766501,-0.344643,-1.031310,-0.320412,-0.106349,...,-0.016186,0.043630,0.261955,-0.063241,0.133332,0.002204,-0.008711,-0.028490,0.002531,1
96,74,10.wav,0,1.109222,-0.762094,-1.423856,-0.167523,-1.210245,0.545457,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004094,-0.007347,0.048764,0.014632,0.001360,1
97,75,19.wav,0,1.109222,-0.762094,-1.423856,-0.167523,-1.210245,0.545457,0.135286,...,-0.038320,-0.134414,-0.017374,0.073740,0.004094,-0.007347,0.048764,0.014632,0.001360,1
98,76,22.wav,0,-0.699710,1.390987,0.118653,0.272301,0.424683,1.042611,-0.036385,...,0.001135,0.315419,-0.027473,0.047252,0.002135,0.000886,0.001469,-0.010688,-0.001253,0
